In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/sri/Documents/ML/text-summarizer-project'

In [4]:
# Entity - return type of function

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH ):
        

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with followingn info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-04 14:15:51,433: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-04 14:15:51,434: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-04 14:15:51,435: INFO: common: created directory at artifacts]
[2024-08-04 14:15:51,436: INFO: common: created directory at artifacts/data_ingestion]
[2024-08-04 14:15:56,143: INFO: 2556159927: artifacts/data_ingestion/data.zip downloaded with followingn info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AECF:22AFD1:3F161:57A6A:66AF3FBE
Accept-Ranges: bytes
Date: Sun, 04 Aug 2024 08:45:53 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10235-MAA
X-Cache: MIS